In [5]:
import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("marquis03/cats-and-dogs") + "/train"
# path = "/Users/phamdinhtrunghieu/.cache/kagglehub/datasets/marquis03/cats-and-dogs/versions/2/train"

print("Path to dataset files:", path)
print(os.listdir(path))


Path to dataset files: /Users/phamdinhtrunghieu/.cache/kagglehub/datasets/marquis03/cats-and-dogs/versions/2/train
['cat', '.DS_Store', 'classname.txt', 'dog']


In [6]:
import torch
from torch import nn
import torch.nn.functional as f
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

INPUTSIZE = 64

In [7]:
def show_image(img):
  resize_transform = transforms.Resize((INPUTSIZE,INPUTSIZE))

  resized_img = resize_transform(img)

  plt.figure(figsize=(8, 4))

  plt.subplot(1, 2, 1)
  plt.imshow(img)
  plt.title('Original Image')

  plt.subplot(1, 2, 2)
  plt.imshow(resized_img)
  plt.title('Resized Image (224x224)')

  plt.show()

In [13]:
dataset_path = f"{path}"

print(os.listdir(dataset_path))

dataset = []

count_label = 0

number_of_label = 2

labels = ["cat", "dog"]

for label in labels:
  data_path = f"{dataset_path}/{label}"
  for example in os.listdir(data_path):
    img = Image.open(f"{data_path}/{example}")
    # show_image(img)
    transform = transforms.Compose([
                                    transforms.Resize((INPUTSIZE,INPUTSIZE)),
                                    transforms.ToTensor()
                                    ])
    x = 1.0 * transform(img)
    if (x.size()[0] != 3):
      continue
    y = torch.zeros(number_of_label)
    y[count_label] = 1

    # print(x,y)

    dataset.append((x,y))

  count_label += 1


['cat', '.DS_Store', 'classname.txt', 'dog']


In [9]:
import random

random.shuffle(dataset)

In [10]:
max_w, max_h = 0, 0
for x, y in dataset:
  print(x.size())
  print(y.size())
  max_w = max(max_w, x.size()[1])
  max_h = max(max_h, x.size()[2])

torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])
torch.Size([3, 64, 64])
torch.Size([2])


In [11]:
print(max_h, max_w)

64 64


In [21]:
class ClassifierVGG16(nn.Module):
  def __init__(self):
    super().__init__()
    tensor_size = INPUTSIZE
    self.conv11 = nn.Conv2d(3, 64, 3, 1, 1) # 224 / 64
    self.conv12 = nn.Conv2d(64, 64, 3, 1, 1)
    self.maxpool1 = nn.MaxPool2d(2,2) # 112 / 32
    tensor_size /= 2
    
    self.conv21 = nn.Conv2d(64, 128, 3, 1, 1) # 112 / 32
    self.conv22 = nn.Conv2d(128, 128, 3, 1, 1)
    self.maxpool2 = nn.MaxPool2d(2,2) # 56 / 16
    tensor_size //= 2
    
    self.conv31 = nn.Conv2d(128, 256, 3, 1, 1) # 56 / 16
    self.conv32 = nn.Conv2d(256, 256, 3, 1, 1)
    self.conv33 = nn.Conv2d(256, 256, 3, 1, 1)
    self.maxpool3 = nn.MaxPool2d(2,2) # 28 / 8
    tensor_size //= 2
    
    self.conv41 = nn.Conv2d(256, 512, 3, 1, 1) # 28 / 8
    self.conv42 = nn.Conv2d(512, 512, 3, 1, 1)
    self.conv43 = nn.Conv2d(512, 512, 3, 1, 1)
    self.maxpool4 = nn.MaxPool2d(2,2) # 14 / 4
    tensor_size //= 2
    
    self.conv51 = nn.Conv2d(512, 512, 3, 1, 1) # 14
    self.conv52 = nn.Conv2d(512, 512, 3, 1, 1)
    self.conv53 = nn.Conv2d(512, 512, 3, 1, 1)
    self.maxpool5 = nn.MaxPool2d(2,2) # 7 / 2
    tensor_size //= 2
    tensor_size = int(tensor_size)
    
    self.fc1 = nn.Linear(tensor_size * tensor_size * 512, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc2 = nn.Linear(4096, 2)
    

  def forward(self,x):
    x = f.relu(self.conv11(x))
    x = f.relu(self.conv12(x))
    x = self.maxpool1(x)
    # print(x.size())
    
    x = f.relu(self.conv21(x))
    x = f.relu(self.conv22(x))
    x = self.maxpool2(x)
    # print(x.size())
    
    x = f.relu(self.conv31(x))
    x = f.relu(self.conv32(x))
    x = f.relu(self.conv33(x))
    x = self.maxpool3(x)
    # print(x.size())
    
    x = f.relu(self.conv41(x))
    x = f.relu(self.conv42(x))
    x = f.relu(self.conv43(x))
    x = self.maxpool4(x)
    # print(x.size())
    
    x = f.relu(self.conv51(x))
    x = f.relu(self.conv52(x))
    x = f.relu(self.conv53(x))
    x = self.maxpool5(x)
    # print(x.size())

    x = torch.flatten(x)
    
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))

    return f.softmax(x, dim = 0)

In [22]:
def get_label(y):
  res = 0

  for id in range(number_of_label):
    if y[res].item() < y[id].item():
      res = id



  return labels[res]

In [25]:
model = ClassifierVGG16()
# model = VGG16()
print(get_label(model(x)))

import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

loss_calculator = nn.CrossEntropyLoss()

for epoch in range(100):
  number_of_example = len(dataset)
  number_of_correct = 0
  print(f"=====epoch:{epoch}=====")
  for id in range(number_of_example):
    x, y = dataset[id]
    y_hat = model(x)

    # loss = ((y - y_hat) ** 2).mean()
    loss = loss_calculator(y, y_hat)

    if get_label(y_hat) == get_label(y):
      number_of_correct += 1

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    print(f"Epoch {epoch} - {id}: {get_label(y)} - {get_label(y_hat)} || Loss: {loss.item()}")
    print(y, y_hat)


dog
=====epoch:0=====
Epoch 0 - 0: cat - dog || Loss: 0.8138421177864075
tensor([1., 0.]) tensor([0.4994, 0.5006], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 1: cat - dog || Loss: 0.813646137714386
tensor([1., 0.]) tensor([0.4996, 0.5004], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 2: cat - dog || Loss: 0.8132734298706055
tensor([1., 0.]) tensor([0.5000, 0.5000], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 3: cat - cat || Loss: 0.8127418756484985
tensor([1., 0.]) tensor([0.5005, 0.4995], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 4: cat - cat || Loss: 0.8120675086975098
tensor([1., 0.]) tensor([0.5012, 0.4988], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 5: cat - cat || Loss: 0.8115184307098389
tensor([1., 0.]) tensor([0.5017, 0.4983], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 6: cat - cat || Loss: 0.8110581636428833
tensor([1., 0.]) tensor([0.5022, 0.4978], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 7: cat - cat || Loss: 0.8105453848838806
tensor([1., 0.]) tensor([0.5027, 0.4973], grad_fn=<SoftmaxBackward0>)
Epoch 0 - 8

KeyboardInterrupt: 